In [32]:
from dash import Dash
import dash_core_components as dcc
import dash_html_components as html
import dash_bootstrap_components as dbc
import plotly.express as px
import pandas as pd
from dash.dependencies import Input, Output
import plotly.express as px
from datetime import datetime, date


In [33]:
from os import pardir, path, getcwd, path
import sys

# mod_path = path.abspath(path.join(pardir))
pack_global = r'/Users/karimkhalil/Coding/development'
pack_project = path.abspath(path.join(getcwd(), pardir))

dirs = [pack_global,pack_project]


for i in dirs:
    # print(i)
    if i not in sys.path:
        sys.path.append(i)

print(sys.path)

['/Users/karimkhalil/Coding/development/stocksapp/notebooks', '/Users/karimkhalil/Coding/development/stocksapp/stocksappenv/lib/python39.zip', '/Users/karimkhalil/Coding/development/stocksapp/stocksappenv/lib/python3.9', '/Users/karimkhalil/Coding/development/stocksapp/stocksappenv/lib/python3.9/lib-dynload', '', '/Users/karimkhalil/Coding/development/stocksapp/stocksappenv/lib/python3.9/site-packages', '/Users/karimkhalil/Coding/development', '/Users/karimkhalil/Coding/development/stocksapp']


In [34]:
# from mypackages.db_loader.mongo import DB_mongo

from mypackages import DB_mongo
from project_package import get_dropdown_label
from pymongo import MongoClient, ASCENDING , DESCENDING
import pandas as pd

In [35]:
stock = DB_mongo._CLIENT.db_fin
mongo_stock = DB_mongo(stock)
mongo_stock.db.tables

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'db_fin'), 'tables')

In [36]:
tables = mongo_stock.all_tables
print(tables)

['cf', 'prices_bckup', 'company_today', 'mrktcap', 'company_wfinancials', 'prices_n_fin', 'bs', 'fx', 'profile', 'idx_price', 'prices', 'idx_all', 'pnl']


In [37]:
mongo_stock.get_df(mongo_stock.db.idx_all)

,_id,symbol,name,currency,stockExchange,exchangeShortName
0,62cc22fa4c9eba2a564feccc,TYX,Treasury Yield 30 Years,USD,NYBOT,INDEX
1,62cc22fa4c9eba2a564feccd,FVX,Treasury Yield 5 Years,USD,NYBOT,INDEX
2,62cc22fa4c9eba2a564fecce,TNX,Treasury Yield 10 Years,USD,NYBOT,INDEX
3,62cc22fa4c9eba2a564feccf,SPGSCI,S&P GSCI Index,USD,SNP,INDEX
4,62cc22fa4c9eba2a564fecd0,STI,STI Index,SGD,SES,INDEX
...,...,...,...,...,...,...
58,62cc22fa4c9eba2a564fed06,XAX,NYSE AMEX COMPOSITE INDEX,USD,New York Stock Exchange,INDEX
59,62cc22fa4c9eba2a564fed07,MID,S&P MID CAP 400 INDEX,USD,New York Stock Exchange,INDEX
60,62cc22fa4c9eba2a564fed08,NYA,NYSE COMPOSITE (DJ),USD,New York Stock Exchange,INDEX
61,62cc22fa4c9eba2a564fed09,X60.TS,S&P/TSX 60 Index,CAD,Toronto,INDEX


In [38]:
mongo_stock.get_df(mongo_stock.db.profile, Symbol='MSFT')

,_id,Symbol,Price,Beta,VolAvg,MktCap,LastDiv,Range,Changes,companyName,...,zip,DCF_diff,DCF,image,ipoDate,defaultImage,isEtf,isActivelyTrading,isFund,isAdr
0,629c9ee60206e9ef45214eca,MSFT,270.02,0.93616,33000374,2.019488e+12,2.42,246.44-349.67,-4.56,Microsoft Corporation,...,98052-6399,1.54418,271.564,https://financialmodelingprep.com/image-stock/...,1986-03-13,False,False,True,False,False


In [39]:
df_idx_all = mongo_stock.get_df(mongo_stock.db.idx_all)
df_idx_price = mongo_stock.get_df(mongo_stock.db.idx_price)
df_idx_price = df_idx_price.merge(df_idx_all[['symbol', 'name']])
idx_sym = mongo_stock.get_unique(mongo_stock.db.idx_price, 'symbol')
# idx_sym = mongo_stock.db.idx_price.distinct('symbol')
df_idx = df_idx_all.loc[df_idx_all['symbol'].isin(idx_sym), ['symbol', 'name' , 'currency', 'stockExchange']]
df_idx[['symbol', 'name']].to_dict('records')
market_idx_dropdown = [{'label': i['name'], 'value': i['symbol']} for i in df_idx[['symbol', 'name']].to_dict('records') ]    
df_idx_info = df_idx_all.loc[df_idx_all['symbol'].isin(idx_sym), ['symbol', 'name']]
fig = px.line(df_idx_price, x='datetime', y = 'adjClose', color='name')

In [40]:
# def get_dropdown_label(some):
#     for i,j in enumerate(market_idx_dropdown):
#         if j['value'] == some:
#             # print(i)
#             index = i
#     # print(index)
#     return market_idx_dropdown[index]['label']

# get_dropdown_label('TYX')

In [41]:
get_dropdown_label(market_idx_dropdown, 'TYX')

'Treasury Yield 30 Years'

In [42]:
df_idx_option = df_idx[df_idx['symbol']=='TYX']
df_idx_option.columns= ['Symbol', 'Name', 'Currency', 'Stock Exchange']
df_idx_option = df_idx_option.transpose().reset_index()
df_idx_option.columns = ["item", "value"]
df_idx_option


,item,value
0,Symbol,TYX
1,Name,Treasury Yield 30 Years
2,Currency,USD
3,Stock Exchange,NYBOT


In [43]:
df_date_check = df_idx_price
df_date_check['datetime'] = df_date_check['datetime'].dt.strftime("%Y-%m-%d")

In [92]:
df_tyx = df_date_check.loc[(df_date_check['datetime'].between( "2020-08-30", "2021-08-30")) & (df_date_check['symbol'] == 'TYX')]
df_tyx.set_index('datetime')

,_id,index,date,open,high,low,close,adjClose,change,changePercent,label,changeOverTime,symbol,volume,unadjustedVolume,vwap,month,year,name
datetime,,,,,,,,,,,,,,,,,,,
2021-08-30,62b067ab1c8a1126546cfd0b,201,2021-08-30,1.926,1.930,1.899,1.900,1.900,-0.026,-1.350,"August 30, 21",-0.01350,TYX,NaN,NaN,NaN,8,2021,Treasury Yield 30 Years
2021-08-27,62b067ab1c8a1126546cfd0c,202,2021-08-27,1.938,1.941,1.916,1.918,1.918,-0.020,-1.032,"August 27, 21",-0.01032,TYX,NaN,NaN,NaN,8,2021,Treasury Yield 30 Years
2021-08-26,62b067ab1c8a1126546cfd0d,203,2021-08-26,1.965,1.977,1.936,1.941,1.941,-0.024,-1.221,"August 26, 21",-0.01221,TYX,NaN,NaN,NaN,8,2021,Treasury Yield 30 Years
2021-08-25,62b067ab1c8a1126546cfd0e,204,2021-08-25,1.922,1.969,1.904,1.959,1.959,0.037,1.925,"August 25, 21",0.01925,TYX,NaN,NaN,NaN,8,2021,Treasury Yield 30 Years
2021-08-24,62b067ab1c8a1126546cfd0f,205,2021-08-24,1.887,1.909,1.885,1.907,1.907,0.020,1.060,"August 24, 21",0.01060,TYX,NaN,NaN,NaN,8,2021,Treasury Yield 30 Years
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-09-04,62b067ab1c8a1126546cfe02,448,2020-09-04,1.384,1.470,1.377,1.470,1.470,0.086,6.214,"September 04, 20",0.06214,TYX,NaN,NaN,NaN,9,2020,Treasury Yield 30 Years
2020-09-03,62b067ab1c8a1126546cfe03,449,2020-09-03,1.386,1.394,1.322,1.341,1.341,-0.045,-3.247,"September 03, 20",-0.03247,TYX,NaN,NaN,NaN,9,2020,Treasury Yield 30 Years
2020-09-02,62b067ab1c8a1126546cfe04,450,2020-09-02,1.422,1.441,1.372,1.375,1.375,-0.047,-3.305,"September 02, 20",-0.03305,TYX,NaN,NaN,NaN,9,2020,Treasury Yield 30 Years


In [94]:
df_tyx.set_index('datetime', inplace=True)
df_tyx['return'] = df_tyx['adjClose'].pct_change(periods=-1)
df_tyx.sort_index()

/var/folders/_7/8tv1x9sn12bcg2fzw5kp9n7h0000gn/T/ipykernel_34841/2770969246.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,_id,index,date,open,high,low,close,adjClose,change,changePercent,label,changeOverTime,symbol,volume,unadjustedVolume,vwap,month,year,name,return
datetime,,,,,,,,,,,,,,,,,,,,
2020-08-31,62b067ab1c8a1126546cfe06,452,2020-08-31,1.516,1.524,1.452,1.452,1.452,-0.064,-4.222,"August 31, 20",-0.04222,TYX,NaN,NaN,NaN,8,2020,Treasury Yield 30 Years,NaN
2020-09-01,62b067ab1c8a1126546cfe05,451,2020-09-01,1.496,1.509,1.421,1.425,1.425,-0.071,-4.746,"September 01, 20",-0.04746,TYX,NaN,NaN,NaN,9,2020,Treasury Yield 30 Years,-0.018595
2020-09-02,62b067ab1c8a1126546cfe04,450,2020-09-02,1.422,1.441,1.372,1.375,1.375,-0.047,-3.305,"September 02, 20",-0.03305,TYX,NaN,NaN,NaN,9,2020,Treasury Yield 30 Years,-0.035088
2020-09-03,62b067ab1c8a1126546cfe03,449,2020-09-03,1.386,1.394,1.322,1.341,1.341,-0.045,-3.247,"September 03, 20",-0.03247,TYX,NaN,NaN,NaN,9,2020,Treasury Yield 30 Years,-0.024727
2020-09-04,62b067ab1c8a1126546cfe02,448,2020-09-04,1.384,1.470,1.377,1.470,1.470,0.086,6.214,"September 04, 20",0.06214,TYX,NaN,NaN,NaN,9,2020,Treasury Yield 30 Years,0.096197
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-08-24,62b067ab1c8a1126546cfd0f,205,2021-08-24,1.887,1.909,1.885,1.907,1.907,0.020,1.060,"August 24, 21",0.01060,TYX,NaN,NaN,NaN,8,2021,Treasury Yield 30 Years,0.018697
2021-08-25,62b067ab1c8a1126546cfd0e,204,2021-08-25,1.922,1.969,1.904,1.959,1.959,0.037,1.925,"August 25, 21",0.01925,TYX,NaN,NaN,NaN,8,2021,Treasury Yield 30 Years,0.027268
2021-08-26,62b067ab1c8a1126546cfd0d,203,2021-08-26,1.965,1.977,1.936,1.941,1.941,-0.024,-1.221,"August 26, 21",-0.01221,TYX,NaN,NaN,NaN,8,2021,Treasury Yield 30 Years,-0.009188


In [109]:
df_tyx.index.max()

Timestamp('2021-08-30 00:00:00')

In [68]:
pd.DataFrame(df_tyx.adjClose.describe())

,adjClose
count,252.000000
mean,1.926925
std,0.311348
min,1.341000
25%,1.654500
50%,1.917000
75%,2.243250
max,2.476000


In [56]:
df_tyx['datetime'].idxmin().strftime("%m/%d/%Y")

AttributeError: 'numpy.int64' object has no attribute 'strftime'

In [55]:
df_tyx[df_tyx['datetime']==df_tyx['datetime'].idxmin().strftime]

,_id,index,date,open,high,low,close,adjClose,change,changePercent,label,changeOverTime,symbol,volume,unadjustedVolume,vwap,datetime,month,year,name
201,62b067ab1c8a1126546cfd0b,201,2021-08-30,1.926,1.93,1.899,1.9,1.9,-0.026,-1.35,"August 30, 21",-0.0135,TYX,NaN,NaN,NaN,2021-08-30,8,2021,Treasury Yield 30 Years


In [ ]:
df_date_check.info()

In [ ]:
d = df_idx_price.to_dict('records')

In [ ]:
for i in d[0]:
    print(i, d[0][i])

In [ ]:
df_fig = df_idx_price
date_min_str = pd.to_datetime(df_fig['datetime'].idxmin()).strftime("%m/%d/%Y")
date_max_str = pd.to_datetime(df_fig['datetime'].idxmax()).strftime("%m/%d/%Y")

date_min = pd.to_datetime(df_fig['datetime'].idxmin()).strftime("%m/%d/%Y")
date_max = pd.to_datetime(df_fig['datetime'].idxmax()).strftime("%m/%d/%Y")

In [ ]:
df_idx_price.date

In [ ]:
df_idx_price['datetime'].dt.strftime("%m-%d-%Y")

In [ ]:
df_idx_price.info()

In [ ]:
df_idx_price['date'].apply(lambda x: x.isoformat())

In [ ]:
cols = [i for i in df_idx_all.columns if i != '_id']

In [ ]:
d =df_fig.to_dict('records')[0]

In [ ]:
type(d['date'])

In [ ]:
pd.to_datetime(df_idx_price['datetime'].idxmax()).strftime("%m/%d/%Y")

In [ ]:
datetime(1995, 8, 5)

In [ ]:
datetime.strptime('2010-01-01', '%Y-%m-%d')

In [ ]:
datetime.strftime(datetime(1995, 8, 5), '%Y-%m-%d')

In [ ]:
type(df_idx_price.to_dict('records'))